<a href="https://colab.research.google.com/github/MostafijurNaeem/R/blob/main/Csv_to_NCDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Step 1: Install the required libraries
!pip install pandas xarray numpy pyproj

# Step 2: Upload the CSV file
from google.colab import files
uploaded = files.upload()  # This will ask you to upload a CSV file

# Step 3: Import necessary libraries
import pandas as pd
import numpy as np
import xarray as xr
from pyproj import CRS

# Step 4: Load the uploaded CSV file
import io
csv_file = next(iter(uploaded))  # Get the name of the uploaded file
df = pd.read_csv(io.BytesIO(uploaded[csv_file]))

# Step 5: Check the column names and print the first few rows
print("Column names in the CSV file:", df.columns)
print("First few rows of the CSV data:")
print(df.head())

# Step 6: Identify latitude, longitude, and the variable column
# Assuming the first two columns are latitude and longitude, and the third column is the variable data
latitude_column = df.columns[0]  # First column (latitude)
longitude_column = df.columns[1]  # Second column (longitude)
variable_column = df.columns[2]  # Third column (variable, e.g., 'sss')

# Step 7: Handle duplicates by aggregating variable data
df_grouped = df.groupby([latitude_column, longitude_column])[variable_column].mean().reset_index()

# Step 8: Reshape the data into a grid (assuming latitudes and longitudes form a grid)
df_sorted = df_grouped.sort_values(by=[latitude_column, longitude_column])

# Create a 2D array for the variable using the sorted data
variable_grid = df_sorted.pivot(index=latitude_column, columns=longitude_column, values=variable_column).values

# Step 9: Create the NetCDF file with dimensions lat, long, and the variable
ds = xr.Dataset(
    {
        variable_column: (['lat', 'long'], variable_grid)
    },
    coords={
        'lat': df_sorted[latitude_column].unique(),
        'long': df_sorted[longitude_column].unique()
    }
)

# Step 10: Add WGS1984 CRS to the NetCDF file
crs = CRS.from_epsg(4326)  # WGS 1984 EPSG code
ds.attrs['crs'] = crs.to_wkt()  # Add CRS as a WKT string

# Step 11: Save the NetCDF file
netcdf_file = 'converted_file_with_CRS.nc'
ds.to_netcdf(netcdf_file)

# Step 12: Provide a download link for the NetCDF file
from google.colab import files
files.download(netcdf_file)  # This will allow you to download the .nc file

# Step 13: Print a success message
print(f"NetCDF file saved and ready to download as {netcdf_file}")


Saving Salinity.csv to Salinity (1).csv
Column names in the CSV file: Index(['latitude', 'longitude', 'sss'], dtype='object')
First few rows of the CSV data:
   latitude  longitude        sss
0     5.125     80.125  33.953490
1     5.125     80.375  33.953490
2     5.125     80.625  33.953490
3     5.125     80.875  33.598530
4     5.125     81.125  33.406742


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

NetCDF file saved and ready to download as converted_file_with_CRS.nc
